<a href="https://colab.research.google.com/github/RohanDubeySF/Training/blob/master/Feature-Selection/Chi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fisher Score- Chisquare Test For Feature Selection
Compute chi-squared stats between each non-negative feature and class.

### This score should be used to evaluate categorical variables in a classification task.
### This score can be used to select the n_features features with the highest values for the test chi-squared statistic from X, which must contain only non-negative features such as booleans or frequencies (e.g., term counts in document classification), relative to the classes.


Recall that the chi-square test measures dependence between stochastic variables, so using this function “weeds out” the features that are the most likely to be independent of class and therefore irrelevant for classification. The Chi Square statistic is commonly used for testing relationships between categorical variables.

It compares the observed distribution of the different classes of target Y among the different categories of the feature, against the expected distribution of the target classes, regardless of the feature categories.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
data=sns.load_dataset('titanic')
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
  data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [ ]:
data.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [ ]:
data=data[['sex','embarked','alone','pclass','survived']]
data

,sex,embarked,alone,pclass,survived
0,male,S,False,3,0
1,female,C,False,1,1
2,female,S,True,3,1
3,female,S,False,1,1
4,male,S,True,3,0
...,...,...,...,...,...
886,male,S,True,2,0
887,female,S,True,1,1
888,female,S,False,3,0
889,male,C,True,1,1


In [ ]:
data['sex']=np.where(data['sex']=='male',1,0)

In [ ]:
ordinal_label = {k: i for i, k in enumerate(data['embarked'].unique(), 0)}
data['embarked'] = data['embarked'].map(ordinal_label)
data['alone']=np.where(data['alone']==True,1,0)

In [ ]:
data

,sex,embarked,alone,pclass,survived
0,1,0,0,3,0
1,0,1,0,1,1
2,0,0,1,3,1
3,0,0,0,1,1
4,1,0,1,3,0
...,...,...,...,...,...
886,1,0,1,2,0
887,0,0,1,1,1
888,0,0,0,3,0
889,1,1,1,1,1


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data.drop('survived',axis=1),data['survived'],test_size=0.2,random_state=6)
x_train

,sex,embarked,alone,pclass
741,1,0,0,1
577,0,0,0,1
810,1,0,1,3
299,0,1,0,1
655,1,0,0,2
...,...,...,...,...
335,1,0,1,3
365,1,0,1,3
618,0,0,0,2
227,1,0,1,3


In [ ]:
x_train['sex'].unique()

array([1, 0])

In [ ]:
data.isnull().sum()

,0
sex,0
embarked,0
alone,0
pclass,0
survived,0


# perform chi-square test

In [ ]:
from sklearn.feature_selection import chi2
chi=chi2(x_train,y_train)

### chi-square returns 2 valus:- F score and p value

In [ ]:
chi

(array([71.15949144, 10.61372218, 13.61343824, 21.74711979]),
 array([3.29483883e-17, 1.12251499e-03, 2.24572225e-04, 3.11056504e-06]))

In [ ]:
chi


(array([71.15949144, 10.61372218, 13.61343824, 21.74711979]),
 array([3.29483883e-17, 1.12251499e-03, 2.24572225e-04, 3.11056504e-06]))

### First row is F-score more the value more important feature is
### second row is p-value less the value more important feature is

In [ ]:
Pvalue=pd.Series(chi[1])
Pvalue.index=x_train.columns
Pvalue.sort_values(ascending=True)

,0
sex,3.294839e-17
pclass,3.110565e-06
alone,2.245722e-04
embarked,1.122515e-03
